In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg
from scipy.sparse import csr_matrix, lil_matrix
from itertools import product


# Problema: A, T y W

In [4]:
T = np.zeros((15,3))
T[12,0] = 1
T[13,1] = 1
T[14,2] = 1
W = np.zeros((15,21))
for i in range(0,9):
    W[i,i] = 1
    W[i,i+9] = 1
    W[9+i%3,i] = 1
W[14,20]= -1
W[13,19]= -1
W[12,18] = -1

## Gurobi

In [5]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [6]:
# D es una matriz rx2 donde en la primera columna están los valores de D_l y en la segunda los de d_l
def problemaMaestro(D, E, c, A, b):
    model = gp.Model('Maestro')
    
    #Verifica que las entradas estén bien.
    if( D.any() and D.shape[1] != c.size+1 ):
        return('Error en las entradas. D debe ser r x (c.size)')
    
    if( E.any() and E.shape[1] != c.size+1 ):
        return('Error en las entradas. E debe ser s x (c.size)')
    
    if( E.any() == False and E.any() == False ): #Está funcionando.
        x = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS)
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(x[a]*c[a] for a in range(0,c.size)) )
        
        model.addMConstr (A, x = None, sense='=', b = b, name="constr" )
        model.optimize()
        rta = np.array([x[i].x for i in range(0,c.size)])

        return(rta, float('-Inf'), model.pi)
    
    elif( D.any() == False ): #Está funcionando.
        e = E[:,-1]
        E = np.hstack((E[:,:-1], np.ones((E.shape[0],1))))
        varx = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        theta = model.addVars(1, lb = float('-Inf'), vtype = GRB.CONTINUOUS, name = 'theta')

        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(varx[a]*c[a] for a in range(0,c.size)) + theta[0] )

        list = [varx[i] for i in range(0,c.size)]
        model.addMConstr (A, x = list, sense='=', b = b, name="constr" )

        list.append(theta[0])
        model.addMConstr (E, x = list, sense='>=', b = e, name="constrE" ) 

        model.optimize()
        rta = np.array([varx[i].x for i in range(0,c.size)])
        return(rta, theta[0].x, model.pi)
    
    elif( E.any() == False ):
        d = D[:,-1]
        D = D[:,:-1] 

        x = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS)
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(x[a]*c[a] for a in range(0,c.size)) )
        
        model.addMConstr (A, x = None, sense='=', b = b, name="constr" )
        
        model.addMConstr (D, x = None, sense='>=', b = d )
        
        model.optimize()
        rta = np.array([x[i].x for i in range(0,c.size)])

        return(rta, float('-Inf'), model.pi)
    
    else: 
        e = E[:,-1]
        E = np.hstack((E[:,:-1], np.ones((E.shape[0],1))))
        d = D[:,-1]
        D = D[:,:-1] 
        
        varx = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        theta = model.addVars(1, lb = float('-Inf'), vtype = GRB.CONTINUOUS, name = 'theta')
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(varx[a]*c[a] for a in range(0,c.size)) + theta[0] )
        list = [varx[i] for i in range(0,c.size)]
        model.addMConstr (A, x = list, sense='=', b = b, name="constr" )
        model.addMConstr (D, x = list, sense='>=', b = d, name="constrD" )

        list.append(theta[0])
        model.addMConstr (E, x = list, sense='>=', b = e, name="constrE" ) 
        
        model.optimize()
        rta = np.array([varx[i].x for i in range(0,c.size)])
        return(rta, theta[0].x, model.pi)

In [28]:
#Retorna true si x está en K2, false de lo contrario.
def xInK2(x):
    k = 0
    #Itero sobre todas las posibles realizaciones de \zeta:
    for (r1,r2,r3) in rposibles:
        for (d1,d2,d3) in dposibles:
            hact = np.array([r1,r1,r1,r2,r2,r2,r3,r3,r3,d1,d2,d3,r1,r2,r3])
            bact = hact - (T @ x)
            
            model = gp.Model('Auxiliar')

            vary = model.addVars(range(0,q.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
            vmas = model.addVars(range(0,W.shape[0]), lb = 0, vtype = GRB.CONTINUOUS, name = 'vmas')
            vmenos = model.addVars(range(0,W.shape[0]), lb = 0, vtype = GRB.CONTINUOUS, name = 'vmenos')
            
            model.modelSense = GRB.MINIMIZE
            model.setObjective( quicksum(vmas[a] + vmenos[a]  for a in range(0,W.shape[0])) )
            
            Waux = np.hstack( (W, np.identity(W.shape[0]), -np.identity(W.shape[0])) ) #Bien
            
            model.addMConstr( Waux, x=None, sense='=', b = bact, name="constr" )
            
            model.optimize()
            if( model.objval > 0 ):
                return([model.pi, hact] )
                break
            
    return np.array([])

## Escenarios
Creamos 5000 escenarios para el vector $\zeta = (r_1,r_2,r_3,d_1,d_2,d_3)$ de forma uniforme. Pueden tomar valores enteros, $ 20 \leq r_1,r_2,r_3 \leq 30$ y $(d_1,d_2,d_3) \in \{(0,8,25),(1,15,35),(1,11,40),(3,20,55),(4,20,70)\}$. Tenemos entonces $10\times 10\times 10 \times 5 = 5000$ valores posibles para $\zeta$. Cada uno lo tomamos con probabilidad $p = \frac{1}{5000}$.

In [35]:
rposibles = [(6,2,4)]
dposibles = [(0,8,14)]

q = np.append(np.full(9,7),np.zeros(12))
c = np.array([10,15,20])

A = np.array([[0,0,0]])
b = np.array([0])


def Lshaped():
    s= 0; r=0; v=0; K=5000
    #En nuestro caso, A, b, c, W, T y q son constantes. 
    #Lo único que cambia a partir de las realizaciones de \zeta es h_k
    D = np.array([]); E = np.array([])
    parar = False
    
    while(parar == False and v < 3):
        v += 1
        print('######################################################################\n', v ,'\n##################################################################################' )
        (xv,thetav,pi) = problemaMaestro(D, E, c, A, b)
        
        print('\n$$$$$$$$$$$$$$$$$$\n xv:\n',xv,'\n')
        print('\n$$$$$$$$$$$$$$$$$$\n theta:\n',thetav,'\n')

        #Si xv está en K2
        sigma = np.array(xInK2(xv))
        
        print('\n$$$$$$$$$$$$$$$$$$\n Sigma:\n',sigma,'\n')
        
        if sigma.any() == False:
            print('%%%%%%%%%%\n', 'Caso 1','\n%%%%%%%%%%' )
            pi = np.zeros((W.shape[0],K))
            k = 0
            #Itero sobre todas las posibles realizaciones de \zeta:
            for (r1,r2,r3) in rposibles:
                for (d1,d2,d3) in dposibles :
                    hact = np.array([r1,r1,r1,r2,r2,r2,r3,r3,r3,d1,d2,d3,r1,r2,r3])
                    bact = hact - (T @ xv)
                    
                    (y,t,piact) = problemaMaestro(np.array([]),np.array([]),q,W,bact) 
                    pi[:,k] = piact #Revisar
                    k+=1
            
            #Calculo E
            E_s =  np.sum( np.array( [ ( np.transpose(pi[:,k]) @ T ) for k in range(0,K) ] ) )
            #Calculo e
            e_s =  np.sum( np.array( [ ( np.transpose(pi[:,k]) @ hact ) for k in range(0,K) ] ) )
            
            wv = e_s + (E_s@xv)
            
            #Condición de parada
            print('%%%%%%%%%% wv: \n', wv,'\n%%%%%%%%%%')
            print('%%%%%%%%%% theta: \n', thetav,'\n%%%%%%%%%%')

            parar = ( np.all(wv <= thetav) )
            if(parar == False):
                s = s+1
                if E.any() == False:
                    np.hstack( (E_s, e_s) )
                else:
                    E = np.vstack( E, np.hstack( (E_s, e_s) ) )
            
        else:
            print('hact', sigma[1])
            D_r = np.transpose(sigma[0])@T
            d_r = np.transpose(sigma[0])@sigma[1]
            r+=1
            if D.any() == False:
                D = np.array([np.hstack( (D_r, d_r) )]) 
            else:
                D = np.vstack( ( D, np.hstack( (D_r, d_r) ) ) )
        
    print( xv )

In [36]:
Lshaped()

######################################################################
 1 
##################################################################################
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x160e05d0
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00

$$$$$$$$$$$$$$$$$$
 xv:
 [0. 0. 0.] 


$$$$$$$$$$$$$$$$$$
 theta:
 -inf 

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 